In [1]:
import cv2
import numpy as np
import os

In [2]:
# Función para obtener las rutas de las imágenes en una carpeta
def obtener_rutas_imagenes(carpeta):
    rutas = []
    for archivo in os.listdir(carpeta):
        ruta = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta):
            rutas.append(ruta)
    return rutas

In [5]:
# Carpeta con imágenes positivas (peatones)
positivas_folder = "dataset/ped-example"
# Carpeta con imágenes negativas (no peatones)
negativas_folder = "dataset/non-ped-example"

# Leer las imágenes positivas y negativas
positivas_rutas = obtener_rutas_imagenes(positivas_folder)
negativas_rutas = obtener_rutas_imagenes(negativas_folder)

# Crear una lista de etiquetas para las imágenes (1 para positivas, 0 para negativas)
etiquetas = np.concatenate((np.ones(len(positivas_rutas)), np.zeros(len(negativas_rutas))), axis=0)

# Crear una lista de rutas de imágenes para entrenamiento
rutas_entrenamiento = positivas_rutas + negativas_rutas

# Inicializar la lista de descriptores de características (HOG)
descriptores_hog = []

# Crear el objeto HOG
hog = cv2.HOGDescriptor()

# Extraer los descriptores HOG de cada imagen de entrenamiento
for ruta in rutas_entrenamiento:
    imagen = cv2.imread(ruta)
    gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    descriptor = hog.compute(gray)
    descriptor = descriptor.flatten() # Aplanar el descriptor
    descriptores_hog.append(descriptor)

# Obtener la longitud máxima del descriptor
max_length = max(len(descriptor) for descriptor in descriptores_hog)

# Rellenar los descriptores más cortos con ceros hasta la longitud máxima
descriptores_hog = [np.pad(descriptor, (0, max_length - len(descriptor)), mode='constant') for descriptor in descriptores_hog]

# Convertir la lista de descriptores y etiquetas a un formato adecuado para SVM
descriptores_hog = np.float32(descriptores_hog)
etiquetas = np.int32(etiquetas)

# Crear el objeto SVM
svm = cv2.ml.SVM_create()

# Establecer el tipo de SVM (en este caso, SVM de tipo C-SVC)
svm.setType(cv2.ml.SVM_C_SVC)

# Establecer el kernel SVM (en este caso, kernel lineal)
svm.setKernel(cv2.ml.SVM_LINEAR)

# Entrenar el modelo SVM
svm.train(descriptores_hog, cv2.ml.ROW_SAMPLE, etiquetas)

# Guardar el modelo entrenado en un archivo
svm.save("modelo_svm.xml")

In [13]:
# Abrir el archivo de video
#cap = cv2.VideoCapture('video/crosswalk.avi')
cap = cv2.VideoCapture('video/fourway.avi')

# Definir los puntos de referencia de la ROI para detección de carriles
#ROI_vertices = [(0, 720), (1280, 720), (800, 450), (580, 450)]
height, width = cap.read()[1].shape[:2]
ROI_vertices = [(200, height), (1100,height), (550, 250)]


svm = cv2.ml.SVM_load('modelo_svm.xml')  # Cargar el modelo entrenado del archivo


def draw_lines(img, lines):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), thickness=2)


# Definir la función para procesar cada cuadro del video
def process_frame(frame):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Crear el objeto HOG
    hog = cv2.HOGDescriptor()

    # Calcular el descriptor HOG de la imagen
    descriptor = hog.compute(gray).reshape(1, -1)

    # Convertir el descriptor a tipo CV_32F
    descriptor = descriptor.astype(np.float32)

    # Normalizar el descriptor
    descriptor /= np.linalg.norm(descriptor)

    # Realizar la predicción utilizando el modelo SVM
    _, result = svm.predict(descriptor)

    # Obtener las coordenadas del rectángulo del peatón detectado
    x, y, w, h = result.ravel()

    # Dibujar un rectángulo alrededor del peatón detectado
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), thickness=2)

    return frame

# Procesar cada cuadro del video y mostrar la imagen de salida
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        line_frame = process_frame(frame)
        cv2.imshow('Video', line_frame)

        # Salir al presionar la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/ml/src/svm.cpp:2013: error: (-215:Assertion failed) samples.cols == var_count && samples.type() == CV_32F in function 'predict'
